In [1]:
import pandas as pd

In [2]:
districts = pd.read_csv('data/districts.csv')
# officeholders = pd.read_csv('data/officeholders.csv')
# candidates = pd.read_csv('data/candidates_dcboe.csv')

In [3]:
def build_commissioner_table(smd):
    
    officeholders = pd.read_csv('data/officeholders.csv')
    
    current_officeholder = officeholders[officeholders['smd'] == smd]
    
    if len(current_officeholder) == 0:
        commissioner_table = '<p><em>Office is vacant.</em></p>'

    else:
    
        commissioner_table = """
            <table border="1">
              <tbody>
              """
        
        commissioner_table += """
            <tr>
              <th>Name</th>
              <td>{full_name}</td>
            </tr>        
        """.format(full_name=current_officeholder['full_name'].values[0])
            
        commissioner_table += f"""
            <tr>
              <th>Email</th>
              <td><a href="mailto:{smd}@anc.dc.gov">{smd}@anc.dc.gov</a></td>
            </tr>
            """

        commissioner_table += """
                </tbody>
            </table>
        """
    
    return commissioner_table

In [4]:
def add_candidates(smd):
    """Add multiple candidates"""
    
    candidates = pd.read_csv('data/candidates_dcboe.csv')
    candidates['source'] = 'DCBOE'
    
    # randomize the order of candidates
    current_candidates = candidates[candidates['smd'] == smd].sample(frac=1)
    
    if len(current_candidates) == 0:
        candidate_block = '<p><em>No known candidates.</em></p>'
        
    else:

        candidate_block = ''

        for idx, cand_row in current_candidates.iterrows():
            candidate_block += build_candidate_table(cand_row)

    return candidate_block


def build_candidate_table(cand_row):
    """Create HTML table for one candidate"""
        
    candidate_table = """
        <table border="1">
          <tbody>
          """

    if 'full_name' in cand_row:
        candidate_table += """
            <tr>
              <th>Name</th>
              <td>{full_name}</td>
            </tr>        
        """.format(full_name=cand_row['full_name'])


    if 'pickup_date' in cand_row:
        candidate_table += """
            <tr>
              <th>Announced</th>
              <td>{pickup_date}</td>
            </tr>        
        """.format(pickup_date=cand_row['pickup_date'])

        
    if 'source' in cand_row:
        candidate_table += """
            <tr>
              <th>Source</th>
              <td>{source}</td>
            </tr>        
        """.format(source=cand_row['source'])


    candidate_table += """
            </tbody>
        </table>
        <br>
    """
    
    return candidate_table

In [12]:
for idx, district in districts.iterrows():
    
    if district['smd'] not in ('1A02', '1A06', '1B05'):
        continue
    
    with open('templates/smd.html') as f:
        output = f.read()
        
    output = output.replace('<!-- replace with smd -->', district['smd'])
    
    output = output.replace('<!-- replace with commissioner table -->', build_commissioner_table(district['smd']))
    
    output = output.replace('<!-- replace with candidate table -->', add_candidates(district['smd']))
    
    output = output.replace('REPLACE_WITH_ANC', district['anc'])
    output = output.replace('REPLACE_WITH_WARD', str(district['ward']))

    soup = BeautifulSoup(output, 'html.parser')
    output_pretty = soup.prettify()
    with open('docs/districts/{smd}.html'.format(smd=district['smd']), 'w') as f:
        f.write(output_pretty)


In [6]:
output

"<html>\n<head>\n<script src='https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.js'></script>\n<link href='https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.css' rel='stylesheet' />\n</head>\n\n<body>\n\n\n<h1>District 1B05</h1>\n\n<div id='map' style='width: 400px; height: 300px;'></div>\n<script>\nmapboxgl.accessToken = 'pk.eyJ1IjoiZGV2aW5icmFkeSIsImEiOiJjazB0aWMyb2IwYTJnM2hsb2twZm1namFoIn0.OUOlfILkB77LwrWhj_uqkQ';\nvar map = new mapboxgl.Map({\n    container: 'map',\n    style: 'mapbox://styles/devinbrady/ckcnzjsxe338q1io3vf1fqade', // stylesheet location\n    center: [-77.035, 38.934], // starting position [lng, lat]\n    zoom: 14\n});\n\nmap.addControl(new mapboxgl.NavigationControl());\n</script>\n\n\n<h2>Current Commissioner</h2>\n\n<p><em>Office is vacant.</em></p>\n\n\n<h2>2020 Candidates</h2>\n\n<p><em>No known candidates.</em></p>\n\n<h2>Links</h2>\n<ul>\n    <li>ANC 1B</li>\n    <li>Ward 1</li>\n    <li>Submit new information</li>\n</ul>\n</body>\n\n</html>\n"

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(output, 'html.parser')

In [10]:
soup.prettify()

'<html>\n <head>\n  <script src="https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.js">\n  </script>\n  <link href="https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.css" rel="stylesheet"/>\n </head>\n <body>\n  <h1>\n   District 1B05\n  </h1>\n  <div id="map" style="width: 400px; height: 300px;">\n  </div>\n  <script>\n   mapboxgl.accessToken = \'pk.eyJ1IjoiZGV2aW5icmFkeSIsImEiOiJjazB0aWMyb2IwYTJnM2hsb2twZm1namFoIn0.OUOlfILkB77LwrWhj_uqkQ\';\nvar map = new mapboxgl.Map({\n    container: \'map\',\n    style: \'mapbox://styles/devinbrady/ckcnzjsxe338q1io3vf1fqade\', // stylesheet location\n    center: [-77.035, 38.934], // starting position [lng, lat]\n    zoom: 14\n});\n\nmap.addControl(new mapboxgl.NavigationControl());\n  </script>\n  <h2>\n   Current Commissioner\n  </h2>\n  <p>\n   <em>\n    Office is vacant.\n   </em>\n  </p>\n  <h2>\n   2020 Candidates\n  </h2>\n  <p>\n   <em>\n    No known candidates.\n   </em>\n  </p>\n  <h2>\n   Links\n  </h2>\n  <ul>\n   <li>\n    ANC